<a href="https://colab.research.google.com/github/ChickenRob0t/W_Scrapping_Meteorologia/blob/main/Wunder_g.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4;

!pip install requests;


In [ ]:
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as soup
import requests
import numpy as np
import pandas as pd


## Datos iniciales necesarios para el programa
Aquí se pasa el código que genera en qué fecha estamos, una estación de prueba y los diccionarios con la información de las estaciones

In [ ]:
fecha_actual = datetime.now()                   #Problemas técnicos con la zona horaria para que me entregue el día de hoy siempre
desplazamiento_gdl = timedelta(hours=-6)

date_p= (fecha_actual + desplazamiento_gdl ).strftime('%Y-%m-%d')  #Esto entrega la fecha de hoy, no importa cuando lo corra.

#date_p='2023-02-20' #Este es para un día cualquiera, si se quiere compilar, sólo quitarle el '#' de al principio

estacion_p='Fortin'   #Estación de prueba, se puede cambiar por los valores de Estaciones_dic

Estaciones_dic={'Fortin': ['IZAPOPAN32', 20.63, 103.48],'Guadalupe': ['IZAPOP10', 20.66, 103.42],'VTHTECH': ['IZAPOP1', 20.64, 103.41],'P_delSol': ['IZAPOP11', 20.65, 103.4],
'Techno_park': ['IZAPOPAN27', 20.73, 103.5],'Aviacion': ['IZAPOP6', 20.75, 103.45],'ONTRACKING': ['IZAPOP35', 20.61, 103.49],'Plaza_la_cima': ['IZAPOPAN38', 20.75, 103.43],
'Monserrat': ['IGUADA119', 20.71, 103.3],'Providencia': ['IGUADALA32', 20.7, 103.37],'La_rioja': ['ILOSGA1', 20.57, 103.46], 'Rancho_Codo':['IENCAR2',21.75,102.10],
'Rancho_Mirasol':['IAGUAS12',21.77,102.11], 'Lagos_Aeroclub':['ILAGOS33',21.26,101.94], 'Prot_Civ_SanJ': ['ISANJU139',21.24,102.32], 'Terra_Aguacatera': ['IARAND35', 20.72,102.41],
'Terra_Pandillo':['ITEPAT5', 20.83,102.69], 'CUALTOS_GDL':['ITEPAT3',20.85,102.78], 'Canadas':['IZAPOPAN36', 20.77,103.37], 'Centro_salud_primavera':['IZAPOPAN30', 20.73,103.56],
'DIF_Nextipac': ['INEXTI1', 20.77,103.53], 'Rancho_viejo': ['ITAPALPA4', 19.94, 103.76], 'La_tuna': ['ITAPAL1', 19.97,103.78], 'ProtCiv_Talpa':['ITALPA1',20.38,104.82],
'YLP_CABO':['ICABOC2', 20.49,105.44], 'Club_careyes':['ILAHUE7',19.44,105.03], 'Tecolote_huer':['ILAHUE6',19.31,104.81], 'CIHUA':['ICIHUA2',19.24,104.57]}

Dic_2={'Fortin': ['IZAPOPAN32', 20.63, 103.48],'Guadalupe': ['IZAPOP10', 20.66, 103.42],'VTHTECH': ['IZAPOP1', 20.64, 103.41],'P_delSol': ['IZAPOP11', 20.65, 103.4]}

In [ ]:
def FtoC(F):
  return round((F-32)*5/9,2)   #Función de Farenheit a Celsius

#Llevar el contenido de la página
Aquí se solicita entrar a la página, se toma el texto y busca la tabla (que casualmente es la primera que hay)

In [ ]:
def pag_table(estac,date,df):

  web='https://www.wunderground.com/dashboard/pws/{}/graph/{}/{}/daily'.format(Estaciones_dic[estac][0],date,date)
  request=requests.get(web)

  texto=request.text
  contenido=soup(texto,'html.parser')
  tablas=contenido.find('table',class_='summary-table')

  df=pd.concat([df,tables(tablas,estac,date)])
  return df

#Generar tablas
Aquí utilizando el texto que provee la página generamos las tablas que necesitamos, en el código toma todas las variables que tiene la primera tabla precipitación, temperatura, temperatura de rocío y humedad. Pero únicamente toma las primeras 2 y las hace tabla.

In [ ]:
def tables(tablas,estacion,date):

  tabla=tablas.find('tbody')
  Head=tablas.find('tr')

  #Guardamos los nombres de columnas, filas y los valores de las celdas.
  rows=list(map (lambda x: x.text, tabla.find_all('th')) )
  columnas=list(map (lambda x: x.text, Head.find_all('th')))
  columnas.pop(0)
  celdas=list( map(lambda x: x.text.split()[0]  , tabla.find_all('td') ) )
  celdas=[celdas[i:i+3] for i in range(0,len(celdas),len(columnas))]          #Cada sub-lista es una fila


  tabla_df=pd.DataFrame(celdas,columns=columnas ) #Ponemos el nombre de las filas y las columnas
  tabla_df=tabla_df.set_index(pd.Index(rows))

  tabla_df=tabla_df.replace({'--':np.nan , ',':''}, regex=True)   #Cambiamos los valores de string a float

  tabla_df=tabla_df.astype(float)

  tabla_2=tabla_df.loc[['Temperature']]   #Aquí escogemos únicamente temperatura y precipitación (tabla df tiene todas)
  tabla_2=tabla_2.apply(FtoC)        #Cambiamos a Celsius
  tabla_2[['Prec_High','Prec_Low','Prec_avg']]=tabla_df.loc['Precipitation'] #Agregamos la precipitación a la tabla
  tabla_2['Estacion']=estacion
  tabla_2[['Latitud','Longitud']]= Estaciones_dic[estacion][1:]
  tabla_2['Fecha']=date



  return tabla_2

In [ ]:
df=pd.DataFrame()
pag_table('Fortin',date_p,df)

,High,Low,Average,Prec_High,Prec_Low,Prec_avg,Estacion,Latitud,Longitud,Fecha
Temperature,23.28,9.94,16.28,0.0,NaN,NaN,Fortin,20.63,103.48,2023-01-20


#Parte 1
Implementación de varias estaciones para un día

In [ ]:
#Esta parte del código genera la base de datos del día, intenta generarla, aveces por la página hay algún problema y si lo hay, lo agrega en la lista 'Fallas'
Dia_estac=pd.DataFrame()
Fallas=[]
for estac in Dic_2:
  try:
    Dia_estac=pag_table(estac,date_p,Dia_estac)
  except AttributeError:
    Fallas.append(estac)
    continue


In [ ]:
print(Fallas)
Dia_estac

['Guadalupe']


,High,Low,Average,Prec_High,Prec_Low,Prec_avg,Estacion,Latitud,Longitud,Fecha
Temperature,27.28,15.39,20.50,2.03,NaN,NaN,Fortin,20.63,103.48,2023-07-14
Temperature,32.22,15.56,22.61,2.80,NaN,NaN,VTHTECH,20.64,103.41,2023-07-14
Temperature,28.17,15.50,21.17,1.51,NaN,NaN,P_delSol,20.65,103.40,2023-07-14


In [ ]:
Dia_estac.to_csv('{}_Wunder_Ground.csv'.format(date_p))  #Genera un csv que se puede descargar, al final podemos subir todos los csv y pegarlos para cada mes.

#**Parte 2**
##Implementación para todo el mes
Aquí hacemos unas funciones extra para que funcione generar una tabla de un mes completo con todas las estaciones. Debido a la cantidad de estaciones que ahora tiene el diccionario, no es recomendable intentarlo cx, quizás se pueda hacer un sub-diccionario donde tarde menos el generar esto.


In [ ]:
#Generamos las fechas de todo el mes que se nos pida:

def mes_list(año,mes):

  fecha_inicio = pd.Timestamp(year=año, month=mes, day=1)
  fecha_fin = fecha_inicio + pd.offsets.MonthEnd()
  rango_fechas = pd.date_range(fecha_inicio, fecha_fin, freq='D')


  # Obtener la lista con cada uno de las fechas (estas a su vez, ayudan a buscar la información en las páginas web)
  return rango_fechas.strftime('%Y-%m-%d').tolist() #Para tenerlo con todo mes, día y año

In [ ]:
#Aquí es donde encontramos para las estaciones de mes_lis todos los días de un mes y lo acoplamos en
mes_lis=mes_list(2023,1)
Mes_completo=pd.DataFrame()

for day in mes_lis:
  for esta in Estaciones_dic:
    Mes_completo=pag_table(esta,day,Mes_completo)


In [ ]:
Mes_completo

,High,Low,Average,Prec_High,Prec_Low,Prec_avg,Estacion,Latitud,Longitud,Fecha
Temperature,21.78,9.83,15.06,0.00,NaN,NaN,Fortin,20.63,103.48,2023-01-01
Temperature,22.94,10.72,16.50,0.00,NaN,NaN,Guadalupe,20.66,103.42,2023-01-01
Temperature,25.00,9.44,17.17,0.01,NaN,NaN,VTHTECH,20.64,103.41,2023-01-01
Temperature,22.61,11.39,16.94,0.00,NaN,NaN,P_delSol,20.65,103.40,2023-01-01
Temperature,22.50,8.00,15.67,0.00,NaN,NaN,Techno_park,20.73,103.50,2023-01-01
Temperature,23.72,9.78,16.67,0.00,NaN,NaN,Aviacion,20.75,103.45,2023-01-01
Temperature,22.50,8.67,15.56,0.00,NaN,NaN,ONTRACKING,20.61,103.49,2023-01-01
Temperature,23.44,12.39,16.94,0.00,NaN,NaN,Plaza_la_cima,20.75,103.43,2023-01-01


##Fallas y pruebas
Aveces esto no termina de funcionar bien, así que este lugar es para pedir una página web exacta y ver qué anda mal

In [ ]:
#Esta celda sólo me ayuda para ver el link de una estación en especial (Por si hay algún problema o algo)
Estaciones_list=list(Estaciones_dic.keys())

web='https://www.wunderground.com/dashboard/pws/{}/graph/{}/{}/daily'.format(Estaciones_dic['Plaza_la_cima'][0],date_p,date_p)
web


'https://www.wunderground.com/dashboard/pws/IZAPOPAN38/graph/2023-06-22/2023-06-22/daily'